In [1]:
import os
import chardet
import pandas as pd
import numpy as np
import glob 

In [2]:
# Cargue de datos
df = pd.read_csv("scopus_bd.csv")

C:\Users\PC1\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (20,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,Authors,Author(s)_ID,Title,Year,Source_title,Volume,Issue,Art._No.,Page_start,Page_end,...,Index_Keywords,ISSN,ISBN,CODEN,PubMed_ID,Document_Type,Publication_Stage,Access_Type,Source,EID
0,Vanzolini P.E.,6602373241;,"On Gymnodactylus amarali Barbour, 1925, with t...",2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,595,611,...,animal; article; Brazil; classification; compa...,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044450573
1,De Wagener A.L.R.,7004422261;,Constraints to the implementation of effective...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,613,623,...,article; developing country; environmental mon...,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044447222
2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600;7202349172;11738879800;6603689065;...,Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,728,...,article; Butia,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319
3,Mulloy B.,55179052500;,The specificity of interactions between protei...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,651,664,...,antithrombin; dermatan sulfate; fucoidin; grow...,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044438562
4,Violanie-Carvalho N.,11739397200;,On the retrieval of significant wave heights f...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,745,755,...,article,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044438196


In [4]:
# dimensión de la matriz
df.shape

(52002, 28)

In [5]:
# Se cambia los punto y comas por comas
df['Author(s)_ID'] = df['Author(s)_ID'].str.replace(';', ',', regex=False)

In [6]:
# Se elimina la última com de la columna la identificación de autores
df['Author(s)_ID'] = df['Author(s)_ID'].map(lambda x: str(x)[:-1])

In [7]:
# Validación que no hayan comas al final de la celda
df['Author(s)_ID'].apply(lambda x: x.count(','))

0        0
1        0
2        6
3        0
4        0
5        1
6        1
7        1
8        3
9        9
10       5
11       0
12       0
13       4
14       2
15       5
16       0
17       1
18       3
19       2
20       3
21       1
22       4
23       0
24       1
25       7
26       4
27       1
28       3
29       1
        ..
51972    1
51973    0
51974    4
51975    3
51976    2
51977    2
51978    2
51979    1
51980    0
51981    2
51982    2
51983    2
51984    0
51985    1
51986    1
51987    0
51988    2
51989    0
51990    1
51991    2
51992    3
51993    0
51994    0
51995    3
51996    1
51997    3
51998    1
51999    3
52000    2
52001    0
Name: Author(s)_ID, Length: 52002, dtype: int64

In [8]:
# Validación de la eliminación de la coma al final de la fila
df['Author(s)_ID']

0                                               6602373241
1                                               7004422261
2        11739855600,7202349172,11738879800,6603689065,...
3                                              55179052500
4                                              11739397200
5                                    6701677981,6602294711
6                                    6602394949,7003428654
7                                    6602000969,7006030716
8             35596061600,9235581600,6602146540,7005645520
9        6701688719,6506023650,8694684200,7801525393,86...
10       55346481900,7004711891,7003857761,6507709088,8...
11                                              6602383159
12                                              7102451322
13       7004019718,7404710202,7003799050,7203013742,35...
14                        6603017420,7004065754,7005887658
15       8366305600,6505949953,8366305200,6603588535,70...
16                                             266429824

In [9]:
# Selección de las columnas de autores
df2 = df[['Authors','Author(s)_ID']]

In [10]:
# Se borra el indice, para que quede como una variable que después pueda servir para agregar los datos
# transformados al df original
df2 = df2.reset_index()

In [11]:
# Se descomprimen las celdas de autores y de la id autores
df3 = (df2.set_index(['index'])
        .stack()
        .str.split(',', expand=True)
        .stack()
        .unstack(-2)
        .reset_index(-1, drop=True)
        .reset_index())

In [12]:
df3

,index,Authors,Author(s)_ID
0,0,Vanzolini P.E.,6602373241
1,1,De Wagener A.L.R.,7004422261
2,2,Mexias A.S.,11739855600
3,2,Berger G.,7202349172
4,2,Gomes M.E.B.,11738879800
5,2,Formoso M.L.L.,6603689065
6,2,Dani N.,6507369527
7,2,Frantz J.C.,11739533300
8,2,Bongiolo E.M.,11739290800
9,3,Mulloy B.,55179052500


In [13]:
# Conteo de id autores unicos, cuidado que hay algunos que no tienen id.
lista = df3['Author(s)_ID'].unique()
len(lista)

113949

In [14]:
lista

array(['6602373241', '7004422261', '11739855600', ..., '6603505857',
       '6701415593', '6701769573'], dtype=object)

In [15]:
# Conteo de autores unicos, cuidado que hay algunos que no tienen nombre
# Tener en cuenta que es mayor que la lista 1, por que un mismo autor puede tener varios alias
lista2 = df3['Authors'].unique()
len(lista2)

117771

In [16]:
lista2

array(['Vanzolini P.E.', 'De Wagener A.L.R.', 'Mexias A.S.', ...,
       ' Bosch J.M.', ' van Wyk D.B.', 'Lambourne J.J.'], dtype=object)

In [17]:
# Tabla que cuenta los datos unicos en el df4
tabla = df3.groupby('Author(s)_ID').agg(['count', 'size', 'nunique']).stack()
tabla

index  Authors
Author(s)_ID                                   
1                       count        2        2
                        size         2        2
                        nunique      2        1
10038943700             count        1        1
                        size         1        1
                        nunique      1        1
10039167100             count        1        1
                        size         1        1
                        nunique      1        1
10039295200             count        3        3
                        size         3        3
                        nunique      3        1
10039317900             count        1        1
                        size         1        1
                        nunique      1        1
10039323700             count        1        1
                        size         1        1
                        nunique      1        1
10039323800             count        1        1
                        size         1        1
                        nunique      1        1
10039361600             count        2        2
                        size         2        2
                        nunique      2        1
10039420200             count        1        1
                        size         1        1
                        nunique      1        1
10039606400             count        1        1
                        size         1        1
                        nunique      1        1
...                                ...      ...
9943569500              count        1        1
                        size         1        1
                        nunique      1        1
9943601200              count        1        1
                        size         1        1
                        nunique      1        1
9943610600              count        3        3
                        size         3        3
                        nunique      3        1
9943655600              count        3        3
                        size         3        3
                        nunique      3        1
9943752800              count        1        1
                        size         1        1
                        nunique      1        1
9943818000              count        1        1
                        size         1        1
                        nunique      1        1
9943898700              count        1        1
                        size         1        1
                        nunique      1        1
9943943000              count        1        1
                        size         1        1
                        nunique      1        1
9943995500              count        1        1
                        size         1        1
                        nunique      1        1
[No author id available count      390      390
                        size       390      390
                        nunique    390        1

[341844 rows x 2 columns]

In [18]:
# Validación de que los autores y la id correspondadn a la misma persona
df3[df3['Author(s)_ID']=='10039361600']

,index,Authors,Author(s)_ID
54750,14242,Velásquez J.C.,10039361600
55345,14428,Velásquez J.C.,10039361600


In [19]:
# Validación de que los autores y la id correspondadn a la misma persona 2
df3.agg(['count', 'size', 'nunique'])

,index,Authors,Author(s)_ID
count,177350,177350,177132
size,177350,177350,177350
nunique,52002,117771,113948


In [20]:
# Tabla que resume cuantos autores hay por id
tabla = df3.groupby('Author(s)_ID').agg(['count', 'size', 'nunique']).sort_values(by=('Authors','nunique'), ascending=False)
tabla

index              Authors             
                        count size nunique   count size nunique
Author(s)_ID                                                   
57201979389                26   26      26      26   26      13
7006347608                 31   31      31      31   31      11
26649280600                28   28      28      28   28      11
24398551800                41   41      41      41   41      10
24780351500                10   10      10      10   10       9
7006056546                 28   28      28      28   28       9
55383143500                15   15      15      15   15       8
56013850700                10   10      10      10   10       8
24278943600                17   17      17      17   17       8
7006271427                 20   20      20      20   20       8
6602255858                  8    8       8       8    8       8
7004361797                 17   17      17      17   17       8
56786625500                12   12      12      12   12       8
6602752010                 23   23      23      23   23       8
12785846200                11   11      11      11   11       7
55962548100                25   25      25      25   25       7
55989702000                27   27      27      27   27       7
56116384000                 8    8       8       8    8       7
57193328371                17   17      17      17   17       7
35606201500                13   13      13      13   13       7
57202771208                30   30      30      30   30       7
55963773400                 9    9       9       9    9       7
35584092200                19   19      19      19   19       7
15036253200                 9    9       9       9    9       7
57191398639                47   47      47      47   47       7
56692945900                 8    8       8       8    8       7
26641319000                 8    8       8       8    8       7
55434810600                20   20      20      20   20       7
6602595777                 15   15      15      15   15       7
55184848400                12   12      12      12   12       7
...                       ...  ...     ...     ...  ...     ...
55822163700                 1    1       1       1    1       1
55822008500                 1    1       1       1    1       1
55821990100                 1    1       1       1    1       1
55823205400                 1    1       1       1    1       1
55823281200                 1    1       1       1    1       1
55826604400                 1    1       1       1    1       1
55823918600                 1    1       1       1    1       1
55826483800                 2    2       2       2    2       1
55826101300                 1    1       1       1    1       1
55826100000                 1    1       1       1    1       1
55825968700                 1    1       1       1    1       1
55825778900                 1    1       1       1    1       1
55825525700                 2    2       2       2    2       1
55825460500                 1    1       1       1    1       1
55825303200                 1    1       1       1    1       1
55824944900                 1    1       1       1    1       1
55824317300                 1    1       1       1    1       1
55823814900                 3    3       3       3    3       1
55823282300                 1    1       1       1    1       1
55823759300                 1    1       1       1    1       1
55823621400                 1    1       1       1    1       1
55823542700                 1    1       1       1    1       1
55823519500                 1    1       1       1    1       1
55823500900                 1    1       1       1    1       1
55823458100                 1    1       1       1    1       1
55823456900                 2    2       2       2    2       1
55823391100                 1    1       1       1    1       1
55823363000                 1    1       1       1    1       1
55823340200                 1    1       1       1    1 

In [21]:
# Validación de que los autores y la id correspondadn a la misma persona 3
df3[df3['Author(s)_ID']=='10040051200']

,index,Authors,Author(s)_ID
99988,28819,Fourie D.,10040051200
100454,28987,Fourie D.,10040051200


In [22]:
# Validación de que los autores y la id correspondadn a la misma persona 4
df3[df3['Author(s)_ID']=='9942560200']

,index,Authors,Author(s)_ID
133101,37732,Urdaneta-Rincón M.,9942560200
133297,37772,Urdaneta-Rincón M.,9942560200
134006,37927,Urdaneta-Rincón M.,9942560200


In [23]:
# Validación de que los autores y la id correspondadn a la misma persona 5
# Como puede verse, el autor puede tener varios alias
df3[df3['Author(s)_ID']=='57201979389']

,index,Authors,Author(s)_ID
46627,12173,Mejía-De Gutiérrez R.,57201979389
46702,12194,Mejía-De Gutiérrez R.,57201979389
46812,12229,Mejía-de Gutiérrez R.,57201979389
47179,12335,Mejía-de Gutiérrez R.,57201979389
48654,12780,De Gutiérrez R.M.,57201979389
49468,13052,de Gutiérrez R.M.,57201979389
71764,19787,Mejía-de-Gutierrez R.,57201979389
72308,19956,de Gutiérrez R.M.,57201979389
72335,19964,de Gutierrez R.M.,57201979389
72408,19989,de Gutiérrez R.M.,57201979389


In [24]:
# Validación de que los autores y la id correspondadn a la misma persona 6
# Se puede concluir que el código parte bien los autores y sus respectivos id
# Por lo que solo habrá que tomar el primer dato de Authors para unificar los datos del df
df3[df3['Author(s)_ID']=='7006347608']

,index,Authors,Author(s)_ID
45671,11885,de Mello J.C.C.B.S.,7006347608
45822,11931,de Mello J.C.C.B.S.,7006347608
46668,12183,Soares-De Mello J.C.C.B.,7006347608
61901,16473,De Mello J.C.C.B.S.,7006347608
62117,16555,De Mello J.C.C.B.S.,7006347608
70585,19374,De Mello J.C.S.,7006347608
71018,19527,De Mello J.C.C.B.S.,7006347608
76945,21426,Soares De Mello J.C.C.B.,7006347608
119345,34125,De Mello J.C.C.B.S.,7006347608
119363,34131,de Mello J.C.C.B.S.,7006347608


In [25]:
# Validación de que los autores y la id correspondadn a la misma persona 7
df3[df3['Author(s)_ID']=='57191398639']

,index,Authors,Author(s)_ID
78784,21904,Müller A.J.,57191398639
81899,22792,Müller A.J.,57191398639
136419,38540,Müller A.J.,57191398639
136942,38691,Müller A.,57191398639
137181,38764,Müller A.J.,57191398639
137260,38785,Müller A.J.,57191398639
142228,40384,Müller A.J.,57191398639
142361,40415,Müller A.J.,57191398639
142406,40428,Müller A.J.S.,57191398639
142422,40434,Müller S. A.J.,57191398639


In [26]:
# Se copia el df para asegurarse que las modificaciones no alteren el df
df4 = df3.copy()

In [27]:
# Se hace un reordenamiento por autores y luego por id
df4.sort_values('Authors').sort_values('Author(s)_ID')

,index,Authors,Author(s)_ID
156686,44559,Anon,1
156704,44572,Anon,1
173426,50636,Mutikanga H.E.,10038943700
35581,9087,Rao K.V.N.S.,10039167100
50725,13392,Zhu Y.,10039295200
51560,13545,Zhu Y.,10039295200
51225,13484,Zhu Y.,10039295200
106326,31008,Hugen R.G.,10039317900
86041,24022,Apolinário J.A.,10039323700
35466,9059,Lopes C.N.,10039323800


In [28]:
# Filtrando los no nulos, en total son 218 filas
df4[df4['Author(s)_ID'].isnull()]

,index,Authors,Author(s)_ID
3301,1018,Scheffer J.P.,NaN
3302,1018,Oliveira A.L.,NaN
3324,1022,Guillon R.,NaN
3413,1038,Rodrigues J.L.,NaN
3606,1065,Caramaschi U.,NaN
3950,1128,Okumura M.,NaN
4124,1155,Wattam A.R.,NaN
5749,1455,Jr.,NaN
7246,1752,Baddini-Martinez J.,NaN
7608,1819,Clemente C.A.,NaN


In [29]:
# Se hace tabla según los id, para poder identificar el primer nombre
tabla2 = df4['Authors'].groupby(df4['Author(s)_ID']).unique().apply(pd.Series)
tabla2

,0,1,2,3,4,5,6,7,8,9,10,11,12
Author(s)_ID,,,,,,,,,,,,,
1,Anon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10038943700,Mutikanga H.E.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10039167100,Rao K.V.N.S.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10039295200,Zhu Y.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10039317900,Hugen R.G.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10039323700,Apolinário J.A.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10039323800,Lopes C.N.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10039361600,Velásquez J.C.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10039420200,Trigueiro J.P.C.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# Se vuelve el primer campo de la tabla para poder seleccionar un único nombre
df6 = pd.DataFrame(tabla2[0]).reset_index()

In [31]:
# Cambio de nombres de columna
df6.columns = ['Author(s)_ID_Rev','Authors_Rev']
# Se crea otra columna con el mismo nombre para poder hacer el merge
df6['Author(s)_ID'] = df6['Author(s)_ID_Rev']
df6

,Author(s)_ID_Rev,Authors_Rev,Author(s)_ID
0,1,Anon,1
1,10038943700,Mutikanga H.E.,10038943700
2,10039167100,Rao K.V.N.S.,10039167100
3,10039295200,Zhu Y.,10039295200
4,10039317900,Hugen R.G.,10039317900
5,10039323700,Apolinário J.A.,10039323700
6,10039323800,Lopes C.N.,10039323800
7,10039361600,Velásquez J.C.,10039361600
8,10039420200,Trigueiro J.P.C.,10039420200
9,10039606400,Bartolomé Á.S.,10039606400


In [32]:
# Unión del merge de los autores depurados, tomando los valores de la izquierda
df7 = pd.merge(df3, df6, on="Author(s)_ID", how='left')
df7

,index,Authors,Author(s)_ID,Author(s)_ID_Rev,Authors_Rev
0,0,Vanzolini P.E.,6602373241,6602373241,Vanzolini P.E.
1,1,De Wagener A.L.R.,7004422261,7004422261,De Wagener A.L.R.
2,2,Mexias A.S.,11739855600,11739855600,Mexias A.S.
3,2,Berger G.,7202349172,7202349172,Berger G.
4,2,Gomes M.E.B.,11738879800,11738879800,Gomes M.E.B.
5,2,Formoso M.L.L.,6603689065,6603689065,Formoso M.L.L.
6,2,Dani N.,6507369527,6507369527,Dani N.
7,2,Frantz J.C.,11739533300,11739533300,Frantz J.C.
8,2,Bongiolo E.M.,11739290800,11739290800,Bongiolo E.M.
9,3,Mulloy B.,55179052500,55179052500,Mulloy B.


In [33]:
# Validación de los núlos, coincide con el primer filtro de 218 filas
df7[df7['Authors_Rev'].isnull()]

,index,Authors,Author(s)_ID,Author(s)_ID_Rev,Authors_Rev
3301,1018,Scheffer J.P.,NaN,NaN,NaN
3302,1018,Oliveira A.L.,NaN,NaN,NaN
3324,1022,Guillon R.,NaN,NaN,NaN
3413,1038,Rodrigues J.L.,NaN,NaN,NaN
3606,1065,Caramaschi U.,NaN,NaN,NaN
3950,1128,Okumura M.,NaN,NaN,NaN
4124,1155,Wattam A.R.,NaN,NaN,NaN
5749,1455,Jr.,NaN,NaN,NaN
7246,1752,Baddini-Martinez J.,NaN,NaN,NaN
7608,1819,Clemente C.A.,NaN,NaN,NaN


In [34]:
# Se eliminan los valores nulos Author_rev copiando los datos de la columna Author
df7.loc[(pd.isnull(df7.Authors_Rev)), 'Authors_Rev'] = df7.Authors

In [35]:
# Revision de que los nulos desaparezcan de la columna Author_rev
df7[df7['Authors_Rev'].isnull()]

,index,Authors,Author(s)_ID,Author(s)_ID_Rev,Authors_Rev


In [36]:
# Revisión que si hayan copiado bien en la columna Author_rev
df7[df7['Author(s)_ID'].isnull()]

,index,Authors,Author(s)_ID,Author(s)_ID_Rev,Authors_Rev
3301,1018,Scheffer J.P.,NaN,NaN,Scheffer J.P.
3302,1018,Oliveira A.L.,NaN,NaN,Oliveira A.L.
3324,1022,Guillon R.,NaN,NaN,Guillon R.
3413,1038,Rodrigues J.L.,NaN,NaN,Rodrigues J.L.
3606,1065,Caramaschi U.,NaN,NaN,Caramaschi U.
3950,1128,Okumura M.,NaN,NaN,Okumura M.
4124,1155,Wattam A.R.,NaN,NaN,Wattam A.R.
5749,1455,Jr.,NaN,NaN,Jr.
7246,1752,Baddini-Martinez J.,NaN,NaN,Baddini-Martinez J.
7608,1819,Clemente C.A.,NaN,NaN,Clemente C.A.


In [37]:
df8 = df7[['index','Author(s)_ID_Rev' ,'Authors_Rev']]
df8

,index,Author(s)_ID_Rev,Authors_Rev
0,0,6602373241,Vanzolini P.E.
1,1,7004422261,De Wagener A.L.R.
2,2,11739855600,Mexias A.S.
3,2,7202349172,Berger G.
4,2,11738879800,Gomes M.E.B.
5,2,6603689065,Formoso M.L.L.
6,2,6507369527,Dani N.
7,2,11739533300,Frantz J.C.
8,2,11739290800,Bongiolo E.M.
9,3,55179052500,Mulloy B.


In [63]:
# Copia del df original, para no modificarlo y pueda servir de respaldo, en caso de errores
df9 = df.copy()
# Se resetea el indice para usarlo en el merge de los dataframe
df9 = df9.reset_index()
df9

,index,Authors,Author(s)_ID,Title,Year,Source_title,Volume,Issue,Art._No.,Page_start,...,Index_Keywords,ISSN,ISBN,CODEN,PubMed_ID,Document_Type,Publication_Stage,Access_Type,Source,EID
0,0,Vanzolini P.E.,6602373241,"On Gymnodactylus amarali Barbour, 1925, with t...",2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,595,...,animal; article; Brazil; classification; compa...,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044450573
1,1,De Wagener A.L.R.,7004422261,Constraints to the implementation of effective...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,613,...,article; developing country; environmental mon...,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044447222
2,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,article; Butia,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319
3,3,Mulloy B.,55179052500,The specificity of interactions between protei...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,651,...,antithrombin; dermatan sulfate; fucoidin; grow...,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044438562
4,4,Violanie-Carvalho N.,11739397200,On the retrieval of significant wave heights f...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,745,...,article,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044438196
5,5,"Baldani J.I., Baldani V.L.D.","6701677981,6602294711",History on the biological nitrogen fixation re...,2005,Anais da Academia Brasileira de Ciencias,77,3,NaN,549,...,Brazil; genetics; genotype; metabolism; microb...,13765,NaN,NaN,1.61276e+07,Review,Final,NaN,Scopus,2-s2.0-24344510274
6,6,"Kirchgatter K., Del Portillo H.A.","6602394949,7003428654",Clinical and molecular aspects of severe malaria,2005,Anais da Academia Brasileira de Ciencias,77,3,NaN,455,...,animal; article; cell adhesion; erythrocyte; g...,13765,NaN,NaN,1.61276e+07,Article,Final,NaN,Scopus,2-s2.0-24344508199
7,7,"Abad-Franch F., Monteiro F.A.","6602000969,7006030716",Molecular research and the control of Chagas d...,2005,Anais da Academia Brasileira de Ciencias,77,3,NaN,437,...,animal; article; Chagas disease; classificatio...,13765,NaN,NaN,1.61276e+07,Article,Final,NaN,Scopus,2-s2.0-24344484749
8,8,"Graeff F.G., Garcia-Leal C., Del-Ben C.M., Gui...","35596061600,9235581600,6602146540,7005645520",Does the panic attack activate the hypothalami...,2005,Anais da Academia Brasileira de Ciencias,77,3,NaN,477,...,cholecystokinin; hydrocortisone; anxiety; arti...,13765,NaN,NaN,1.61276e+07,Article,Final,NaN,Scopus,2-s2.0-24344458134
9,9,"Hesp P.A., Dillenburg S.R., Barboza E.G., Toma...","6701688719,6506023650,8694684200,7801525393,86...","Beach ridges, foredunes or transgressive dunef...",2005,Anais da Academia Brasileira de Ciencias,77,3,NaN,493,...,NaN,13765,NaN,NaN,NaN,Article,Final,NaN,Scopus,2-s2.0-24344442825


In [64]:
# Unión del df original con el df depurado
df10 = pd.merge(df9, df8, on="index", how='left')
# Hay que volver los números a ; para que pueda recargarse después, primer hay que crearlo como string.
df10['Author(s)_ID'] = df10['Author(s)_ID'].astype(str)
df10

,index,Authors,Author(s)_ID,Title,Year,Source_title,Volume,Issue,Art._No.,Page_start,...,ISBN,CODEN,PubMed_ID,Document_Type,Publication_Stage,Access_Type,Source,EID,Author(s)_ID_Rev,Authors_Rev
0,0,Vanzolini P.E.,6602373241,"On Gymnodactylus amarali Barbour, 1925, with t...",2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,595,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044450573,6602373241,Vanzolini P.E.
1,1,De Wagener A.L.R.,7004422261,Constraints to the implementation of effective...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,613,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044447222,7004422261,De Wagener A.L.R.
2,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,11739855600,Mexias A.S.
3,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,7202349172,Berger G.
4,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,11738879800,Gomes M.E.B.
5,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,6603689065,Formoso M.L.L.
6,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,6507369527,Dani N.
7,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,11739533300,Frantz J.C.
8,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,11739290800,Bongiolo E.M.
9,3,Mulloy B.,55179052500,The specificity of interactions between protei...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,651,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044438562,55179052500,Mulloy B.


In [65]:
# Validación de que si haya hecho la partición de manera adecuada
df10[['Authors','Author(s)_ID','Authors_Rev','Author(s)_ID_Rev']]

,Authors,Author(s)_ID,Authors_Rev,Author(s)_ID_Rev
0,Vanzolini P.E.,6602373241,Vanzolini P.E.,6602373241
1,De Wagener A.L.R.,7004422261,De Wagener A.L.R.,7004422261
2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Mexias A.S.,11739855600
3,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Berger G.,7202349172
4,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Gomes M.E.B.,11738879800
5,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Formoso M.L.L.,6603689065
6,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Dani N.,6507369527
7,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Frantz J.C.,11739533300
8,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Bongiolo E.M.,11739290800
9,Mulloy B.,55179052500,Mulloy B.,55179052500


In [66]:
df10

,index,Authors,Author(s)_ID,Title,Year,Source_title,Volume,Issue,Art._No.,Page_start,...,ISBN,CODEN,PubMed_ID,Document_Type,Publication_Stage,Access_Type,Source,EID,Author(s)_ID_Rev,Authors_Rev
0,0,Vanzolini P.E.,6602373241,"On Gymnodactylus amarali Barbour, 1925, with t...",2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,595,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044450573,6602373241,Vanzolini P.E.
1,1,De Wagener A.L.R.,7004422261,Constraints to the implementation of effective...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,613,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044447222,7004422261,De Wagener A.L.R.
2,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,11739855600,Mexias A.S.
3,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,7202349172,Berger G.
4,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,11738879800,Gomes M.E.B.
5,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,6603689065,Formoso M.L.L.
6,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,6507369527,Dani N.
7,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,11739533300,Frantz J.C.
8,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...","11739855600,7202349172,11738879800,6603689065,...",Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,11739290800,Bongiolo E.M.
9,3,Mulloy B.,55179052500,The specificity of interactions between protei...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,651,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044438562,55179052500,Mulloy B.


In [67]:
# Se hace una conversion de las comas por punto y coma en la id de los autores, para que el valor cuando se abra en
# excel no genere inconsistencia, ni cuando se vaya a cargar otra vez
df10['Author(s)_ID'] = df10['Author(s)_ID'].str.replace(',', ';', regex=False)

In [68]:
df10

,index,Authors,Author(s)_ID,Title,Year,Source_title,Volume,Issue,Art._No.,Page_start,...,ISBN,CODEN,PubMed_ID,Document_Type,Publication_Stage,Access_Type,Source,EID,Author(s)_ID_Rev,Authors_Rev
0,0,Vanzolini P.E.,6602373241,"On Gymnodactylus amarali Barbour, 1925, with t...",2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,595,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044450573,6602373241,Vanzolini P.E.
1,1,De Wagener A.L.R.,7004422261,Constraints to the implementation of effective...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,613,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044447222,7004422261,De Wagener A.L.R.
2,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600;7202349172;11738879800;6603689065;...,Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,11739855600,Mexias A.S.
3,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600;7202349172;11738879800;6603689065;...,Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,7202349172,Berger G.
4,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600;7202349172;11738879800;6603689065;...,Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,11738879800,Gomes M.E.B.
5,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600;7202349172;11738879800;6603689065;...,Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,6603689065,Formoso M.L.L.
6,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600;7202349172;11738879800;6603689065;...,Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,6507369527,Dani N.
7,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600;7202349172;11738879800;6603689065;...,Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,11739533300,Frantz J.C.
8,2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600;7202349172;11738879800;6603689065;...,Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,11739290800,Bongiolo E.M.
9,3,Mulloy B.,55179052500,The specificity of interactions between protei...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,651,...,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044438562,55179052500,Mulloy B.


In [69]:
# Se da un orden de columnas para cuando se cree el archivo csv
columns_save = ['Authors','Author(s)_ID','Authors_Rev','Author(s)_ID_Rev','Title','Year','Source_title','Volume','Issue','Art._No.','Page_start','Page_end','Page_count','Cited_by','DOI','Link','Affiliations','Authors_with_affiliations','Abstract','Author_Keywords','Index_Keywords','ISSN','ISBN','CODEN','PubMed_ID','Document_Type','Publication_Stage','Access_Type','Source','EID']

In [71]:
df10.to_csv("scopus_bd_aumentada.csv",columns=columns_save,sep=',',index=False)